# Kompas

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import random

def filter_ads(soup, ad_classes):
    ad_divs = soup.find_all(class_= ad_classes) 
    for ad_div in ad_divs:
        ad_div.extract()

def filter_text(text):
    unwanted_texts = ['Dapatkan update berita pilihan dan breaking news setiap hari dari Kompas.com. Mari bergabung di Grup Telegram "Kompas.com News Update", caranya klik link https://t.me/kompascomupdate, kemudian join. Anda harus install aplikasi Telegram terlebih dulu di ponsel.', "Baca juga:", "JAKARTA, KOMPAS.com - "]
    
    filtered_text = text
    for unwanted_text in unwanted_texts:
        filtered_text = filtered_text.replace(unwanted_text, '')

    return filtered_text

train_csv_file = open('training_data1.csv', 'w', newline='', encoding='utf-8')
test_csv_file = open('True.csv', 'w', newline='', encoding='utf-8')

train_csv_writer = csv.writer(train_csv_file)
test_csv_writer = csv.writer(test_csv_file)

train_csv_writer.writerow(['Title', 'URL', 'Text', 'Label'])
test_csv_writer.writerow(['Title', 'URL', 'Text'])

num_pages = 800

# List to store scraped data
data = []

# List of target dates
target_dates = ["20 Maret 2024", "14 Februari 2024", 
                "4 Februari 2024", "21 Januari 2024", 
                "7 Januari 2024", "22 Desember 2023", 
                "12 Desember 2023", "25 Oktober 2023", 
                "19 Oktober 2023"]

for page in range(1, num_pages + 1):
    url = f"https://pemilu.kompas.com/news?page={page}"
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    titles = soup.find_all('h3', class_='listTitle')
    links = soup.find_all('a', attrs={'href': re.compile("^http://(?:regional|nasional|megapolitan)")})

    for title, link in zip(titles, links):
        title_text = title.text.strip()
        title_date_element = title.find_next(class_='titleDate')
        if title_date_element:
            title_date = title_date_element.text.strip()
            # print(title_date)
        else:
            continue
        
        url = link.get('href')

        if title_date in target_dates:
            news_page = requests.get(url)
            news_soup = BeautifulSoup(news_page.content, 'html.parser')

            filter_ads(news_soup, 'inner-link-baca-juga')

            text_content = news_soup.find('div', class_='read__content')
            if text_content:
                text = text_content.text.strip()
                text = filter_text(text)
            else:
                text = "News content not found for: " + url

            # Assign 80% of the data to the training set and 20% to the testing set
            if random.random() < 0.8:
                data.append((title_text, url, text, 1))
            else:
                test_csv_writer.writerow([title_text, url, text])

random.shuffle(data)

for item in data:
    train_csv_writer.writerow(item)

train_csv_file.close()
test_csv_file.close()


Half (data terlalu banyak)

In [12]:
import csv
import random

def split_csv(input_file, divide_number):
    with open(input_file, 'r', newline='', encoding='utf-8') as existing_file:
        existing_csv_reader = csv.reader(existing_file)
        header = next(existing_csv_reader)
        data = list(existing_csv_reader)

    random.shuffle(data)
    half_length = len(data) // divide_number
    half = data[:half_length]

    output_file = f'Half/{input_file}'
    
    with open(output_file, 'w', newline='', encoding='utf-8') as first_half_file:
        first_half_csv_writer = csv.writer(first_half_file)
        first_half_csv_writer.writerow(header)
        first_half_csv_writer.writerows(half)

split_csv('True.csv', 2)
split_csv('training_data1.csv', 3)


Clean dataset

In [2]:
import csv
import re

def filter_text(text):
    filtered_text = text

    filtered_text = ' '.join(filtered_text.split())
    filtered_text = re.sub(r'http\S+', '', filtered_text)


    return filtered_text.strip()

def filter_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            for row in reader:
                row[2] = filter_text(row[2])
                writer.writerow(row)

input_train = 'Half/training_data1.csv'
output_train = 'Half_Cleaned/training_data1.csv'
filter_csv(input_train, output_train)
                
input_test = 'Half/True.csv'
output_test = 'Half_Cleaned/True.csv'
filter_csv(input_test, output_test)